# 📈 Stock Movement Prediction Model: K-Nearest Neighbors (KNN)

### 1. Model Overview
This program implements a K-Nearest Neighbors (KNN) machine learning model, aiming to predict the stock price direction (Up/Down) of TSMC (2330) by leveraging multi-dimensional technical indicators, global macro data, and related market information.
- **KNN**: A distance-based classification algorithm with a strong capability to capture non-linear relationships between financial features.

### 2. Feature Engineering
The primary features utilized in this model include:
- **Price and Volume Indicators**: Open, High, Low, Close, Volume, Adj Close.
- **Technical Indicators**: Moving Averages (MA), Relative Strength Index (RSI), and MACD series indicators.
- **Market Interdependency**: TSMC ADR (ADR_Close), Dow Jones Industrial Average (DJI_Close), and Volatility Index (VIX_Close).
- **Preprocessing**: `StandardScaler` standardization is applied to the KNN model to eliminate feature scale differences and ensure distance-based accuracy.

### 3. Requirements
Ensure the following packages are installed before execution:
- `pandas`, `numpy`, `scikit-learn`, `matplotlib`

### 4. File Paths
- **Input**: `../../../data/processed/train.csv` & `test.csv`
- **Output**: `../../../data/results/KNN_pred.csv` (The prediction results will be exported as meta-features for the subsequent Stacking ensemble layer.)

# 📊 Phase 1: Data Acquisition & Environment Setup

This project aims to construct a robust movement prediction model for TSMC (2330) stock. Before commencing feature engineering and model training, we first perform data loading and verify the underlying data structure.

### 1. Data Sources
We have integrated multi-dimensional financial data spanning from 2010 to 2025, totaling sixteen years of trading records:
* **Taiwan Market Trading Data**: Includes price and volume information (Open, Close, High, Low, Volume) for TSMC (2330).
* **Technical Indicators**: The raw dataset contains key quantitative indicators such as Moving Averages (MA), RSI, MACD, and K/D.
* **U.S. Market Interdependency**: Integration of major U.S. indices, including NASDAQ, S&P 500 (SPX), Dow Jones Industrial Average (DJI), and the Philadelphia Semiconductor Index (SOX).
* **External Sentiment Indicators**: TSMC ADR price movements and the VIX Volatility Index.

### 2. Dataset Splitting Principles
* **Training Set**: Utilizes historical data from 2010 to 2024 to build a stable market feature model.
* **Test Set**: Uses 2025 data as an Out-of-Sample test to verify the model's generalization ability on unseen data.

### 3. Objectives
The primary objective of this phase is to establish verified data retrieval paths and perform standardization on time-series fields. This ensures data quality and consistency, laying the critical foundation for subsequent Feature Enhancement.

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import warnings
warnings.filterwarnings('ignore')

In [2]:
train_raw = pd.read_csv("../../../data/processed/train.csv")
test_raw  = pd.read_csv("../../../data/processed/test.csv")

In [3]:
# 如果 Date 是字串，轉成 datetime
if 'Date' in train_raw.columns:
    train_raw['Date'] = pd.to_datetime(train_raw['Date'])
    test_raw['Date']  = pd.to_datetime(test_raw['Date'])

In [4]:
print(train_raw.columns.tolist())

['Date', 'Capacity', 'Turnover', 'Open', 'High', 'Low', 'Close', 'Change', 'Transaction', 'MA5', 'MA10', 'MA5_Capacity', 'BR5', 'BR10', 'High-Low', 'Open-Close', 'EMA5', 'EMA10', 'K', 'D', 'MACD', 'MACD_signal', 'MACD_hist', 'RSI14', 'STD20', 'ADX', 'RSI', 'DJI', 'NASDAQ', 'SOX', 'SPX', 'ADR', 'twclose', 'twopen', 'Movement', 'rate']


## 🛠️ Phase 2: Data Preprocessing & Feature Engineering

This phase transforms raw financial data into features that machine learning models can interpret. We retain fundamental price-volume information and expand the analysis dimensions through a custom `enhance_features` function.

### 1. Core Feature Categories
* **Market Fundamentals**: Includes raw price and volume data such as Open, High, Low, Close, and Volume.
* **Technical Analysis Indicators**: 
    * Trend Indicators: MA5, MA10, EMA, and MACD.
    * Momentum Indicators: RSI, K, D, and ADX.
    * Volatility Indicators: STD20 and Bollinger Bands.
* **Global Market Interdependency**: Integration of major U.S. indices (DJI, NASDAQ, SOX, SPX) and TSMC ADR price movements.
* **Custom Enhanced Features**: 
    * **Lags**: Captures time-series inertia by considering price and momentum changes over the past 1 to 10 days.
    * **Pattern Analysis**: Mathematical representation of candlestick body size, shadow ratios, and specialized price formations.
    * **Continuity Indicators**: Tracks consecutive up/down days to detect extreme market sentiment and potential reversals.

In [5]:
def enhance_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    特徵工程增強函式：
    將原始股價與指標數據轉換為更具預測力的特徵（包含動量、型態、均線交叉、美股聯動等）。
    """
    df = df.copy()
    
    # 確保按日期排序：時間序列模型必須確保過去的資料不會參考到未來（防止資料洩漏）
    if 'Date' in df.columns:
        df = df.sort_values('Date').reset_index(drop=True)

    # ============= 1. 基礎價格與技術指標衍生特徵 =============
    # 計算當日漲跌幅與震幅
    df['Price_momentum'] = df['Close'] / df['Open'] - 1
    df['High_Low_range'] = (df['High'] - df['Low']) / df['Close']

    # 均線交叉特徵：判斷短期趨勢是否強於中期
    df['MA5_MA10_cross']   = (df['MA5'] > df['MA10']).astype(int)
    df['Close_above_MA5']  = (df['Close'] > df['MA5']).astype(int)
    df['Close_above_MA10'] = (df['Close'] > df['MA10']).astype(int)

    # RSI 強弱判斷：定義超買(>70)與超跌(<30)區間
    df['RSI_oversold']   = (df['RSI'] < 30).astype(int)
    df['RSI_overbought'] = (df['RSI'] > 70).astype(int)

    # MACD 趨勢與黃金交叉判斷
    df['MACD_above_signal'] = (df['MACD'] > df['MACD_signal']).astype(int)
    df['MACD_golden_cross'] = (
        (df['MACD'] > df['MACD_signal']) &
        (df['MACD'].shift(1) <= df['MACD_signal'].shift(1))
    ).astype(int)
    df['MACD_positive'] = (df['MACD'] > 0).astype(int)

    # KD 指標交叉與區間判斷 (判斷短期轉折)
    df['K_above_D'] = (df['K'] > df['D']).astype(int)
    df['K_D_golden_cross'] = (
        (df['K'] > df['D']) & (df['K'].shift(1) <= df['D'].shift(1))
    ).astype(int)
    df['K_D_dead_cross'] = (
        (df['K'] < df['D']) & (df['K'].shift(1) >= df['D'].shift(1))
    ).astype(int)
    df['KD_oversold']   = ((df['K'] < 20) & (df['D'] < 20)).astype(int)
    df['KD_overbought'] = ((df['K'] > 80) & (df['D'] > 80)).astype(int)

    # 美股指數聯動：計算道瓊(DJI)、標普500(SPX)、那斯達克(NASDAQ)的當日變動率
    df['DJI_change']    = df['DJI'].pct_change()
    df['SPX_change']    = df['SPX'].pct_change()
    df['NASDAQ_change'] = df['NASDAQ'].pct_change()

    # 相對強弱特徵：個股相對於大盤(twclose)的超額報酬
    df['Stock_vs_TW'] = df['Close'].pct_change() - df['twclose'].pct_change()

    # 滯後項特徵 (Lags)：捕捉歷史數據的慣性（考慮過去 1~10 天的漲跌、漲幅與 RSI）
    for lag in [1, 2, 3, 4, 5, 6, 7, 10]:
        df[f'Movement_lag{lag}'] = df['Movement'].shift(lag)
        df[f'Change_lag{lag}']   = df['Change'].shift(lag)
        df[f'RSI_lag{lag}']      = df['RSI'].shift(lag)
        df[f'Return{lag}']       = df['Close'].pct_change(lag)

    # 波動度特徵：計算 5 日短線與 20 日長線的波動比例
    df['Volatility20']     = df['STD20'] 
    df['Volatility5']      = df['Close'].rolling(5).std()
    df['Volatility_ratio'] = df['Volatility5'] / (df['Volatility20'] + 1e-8)

    # 滾動平均特徵：平滑化近 5 日的表現
    df['Change_mean_5d'] = df['Change'].rolling(5).mean()
    df['Change_std_5d']  = df['Change'].rolling(5).std()
    df['RSI_mean_5d']    = df['RSI'].rolling(5).mean()

    # 計算連續上漲/下跌天數：捕捉極端情緒與反轉可能
    up_mask   = (df['Movement'] == 1).astype(int)
    down_mask = (df['Movement'] == 0).astype(int)
    df['Consecutive_up'] = up_mask.groupby((up_mask == 0).cumsum()).cumsum()
    df['Consecutive_down'] = down_mask.groupby((down_mask == 0).cumsum()).cumsum()

    # ============= 2. K 棒實體與影線分析（價格型態） =============
    price_range = (df['High'] - df['Low']).replace(0, np.nan)
    # 收盤價在當日最高低點的位置：判斷收盤強度（0=最低點, 1=最高點）
    df["close_pos"] = (df["Close"] - df["Low"]) / (price_range + 1e-8)
    # 實體、上影線、下影線比例：辨識壓力與支撐區間
    df["body_size"]    = (df["Close"] - df["Open"]).abs() / (df["Open"] + 1e-8)
    df["upper_shadow"] = (df["High"] - df[["Open", "Close"]].max(axis=1)) / (df["Open"] + 1e-8)
    df["lower_shadow"] = (df[["Open", "Close"]].min(axis=1) - df["Low"]) / (df["Open"] + 1e-8)
    # 漲跌判斷
    df["is_bullish"] = (df["Close"] > df["Open"]).astype(int)
    df["is_bearish"] = (df["Close"] < df["Open"]).astype(int)
    # 大陽線/大陰線辨識
    df["marubozu_like"] = ((df["body_size"] > 0.02) & (df["upper_shadow"] < 0.005) & (df["lower_shadow"] < 0.005)).astype(int)

    # ============= 3. 布林通道 (Bollinger Bands) =============
    # 衡量價格波動範圍與極端超買賣區間
    bb_mid = df["Close"].rolling(20).mean()
    bb_std = df["Close"].rolling(20).std()
    df["BB_mid"]   = bb_mid
    df["BB_std"]   = bb_std
    df["BB_upper"] = bb_mid + 2 * bb_std
    df["BB_lower"] = bb_mid - 2 * bb_std
    # 價格在布林通道中的位置
    df["BB_pos"]   = (df["Close"] - df["BB_lower"]) / ((df["BB_upper"] - df["BB_lower"]).replace(0, np.nan) + 1e-8)

    # ============= 4. 多週期動量特徵 =============
    # 衡量不同時間框架下的報酬率 (3, 5, 10, 20 日)
    df["mom3"]  = df["Close"].pct_change(3)
    df["mom5"]  = df["Close"].pct_change(5)
    df["mom10"] = df["Close"].pct_change(10)
    df["mom20"] = df["Close"].pct_change(20)

    # ============= 5. 長期趨勢均線與黃金/死亡交叉 =============
    df["MA20"]  = df["Close"].rolling(20).mean()
    df["MA60"]  = df["Close"].rolling(60).mean()
    df["MA120"] = df["Close"].rolling(120).mean()
    df["Close_above_MA20"]  = (df["Close"] > df["MA20"]).astype(int)
    df["Close_above_MA60"]  = (df["Close"] > df["MA60"]).astype(int)
    df["Close_above_MA120"] = (df["Close"] > df["MA120"]).astype(int)
    df["MA5_MA20_cross"]  = (df["MA5"] > df["MA20"]).astype(int)
    df["MA20_MA60_cross"] = (df["MA20"] > df["MA60"]).astype(int)

    # ============= 6. 成交量量能特徵 =============
    if 'Volume' in df.columns:
        # 成交量變化率與量能爆發辨識
        df["vol_change"]   = df["Volume"].pct_change()
        vol_ma20           = df["Volume"].rolling(20).mean()
        df["vol_ratio20"]  = df["Volume"] / (vol_ma20 + 1e-8)
        df["vol_spike"]    = (df["vol_ratio20"] > 1.5).astype(int)

    return df

### 3. Feature Set Definition

To capture the spillover effects of the U.S. market on the Taiwan stock market, we defined specific feature subsets for model testing:
* **US_RAW**: Includes the raw closing prices of the four major U.S. indices (NASDAQ, S&P 500, DJI, and SOX) and TSMC ADR.
* **US_CHANGE**: Captures the daily percentage changes of these indices to reflect real-time shifts in market sentiment.
* **US_FEATURES**: Integrates the two categories above to construct a comprehensive U.S. market influence matrix.

In [6]:
US_RAW = ['NASDAQ', 'SPX', 'DJI', 'ADR', 'SOX']
US_CHANGE = ['NASDAQ_change', 'SPX_change', 'DJI_change']

US_FEATURES = US_RAW + US_CHANGE

### 4. Smart NA Imputation

Due to the extensive use of rolling windows and lags in feature engineering, missing values (NaNs) inevitably occur in the early stages of the dataset. To ensure the integrity of model training, we designed the `fill_na_safely` function to perform categorical imputation:

* **Binary Signals**: For features such as moving average crossovers or overbought/oversold signals, missing values are filled with `0`, representing "no specific event occurred."
* **Historical Lags**: Missing values are filled with `-1` to distinguish "absence of historical data" from "normal numerical values."
* **Oscillators and Momentum**: For features such as volatility or Bollinger Band positions, values are filled with `0` to maintain neutral characteristics.
* **Continuous Numerical Data**: `ffill` (forward fill) is applied to ensure the logic aligns with time-series reality, strictly avoiding the use of future information.

In [7]:
def fill_na_safely(df: pd.DataFrame) -> pd.DataFrame:
    """
    智能缺失值填補函式 (Smart NA Imputation)：
    根據金融特徵的物理意義進行分類處理，避免簡單填補造成的資訊扭曲。
    """
    df = df.copy()

    for col in df.columns:
        # 跳過不需填補的欄位（時間、標籤、輔助標籤）
        if col in ['Date', 'Movement', 'Movement_tomorrow', 'is_train']:
            continue

        # ============= 1. 二元訊號與事件類特徵 =============
        # 邏輯：缺值通常發生在資料初期（還沒累積足夠天數），此時應視為「無事件發生」
        if (
            col.endswith('_cross') or      # 均線或指標交叉
            col.endswith('_spike') or      # 量能或波動爆發
            col.endswith('_breakout') or   # 價格突破
            ('MACD_' in col and 'cross' in col) or
            ('K_D_' in col and 'cross' in col) or
            col.endswith('_oversold') or   # 超跌訊號
            col.endswith('_overbought') or # 超買訊號
            col.startswith('is_') or       # 布林/K線狀態判斷
            col.endswith('_above_MA5') or
            col.endswith('_above_MA10') or
            col.endswith('_above_MA20') or
            col.endswith('_above_MA60') or
            col.endswith('_above_MA120') or
            col == 'vol_spike' or
            col == 'marubozu_like'
        ):
            df[col] = df[col].fillna(0) # 0 代表「無訊號」
            continue

        # ============= 2. 滯後類特徵 (Lag Features) =============
        # 邏輯：當資料尚未累積到足夠的 lag 天數時，使用 -1 作為標記。
        # 這能讓模型（特別是 Tree-based 模型）區分「無歷史」與「數值為 0」的差異。
        if 'lag' in col:
            df[col] = df[col].fillna(-1)
            continue

        # ============= 3. 滾動計算與震盪指標 =============
        # 邏輯：這類特徵反映的是相對位置或動量，初始值補 0 維持中性。
        if (
            col.endswith('_5d') or
            col.startswith('Volatility') or
            col.startswith('BB_') or
            col.startswith('mom') or
            col in ['close_pos', 'body_size', 'upper_shadow', 'lower_shadow', 'BB_pos']
        ):
            df[col] = df[col].fillna(0)
            continue

        # ============= 4. 變動率與回報率類型 =============
        # 邏輯：對於變動率類特徵，NaN 通常發生在第一筆，填 0 代表「無變動」。
        if (
            'change' in col.lower() or
            'return' in col.lower() or
            'ratio' in col.lower()
        ):
            df[col] = df[col].fillna(0)
            continue

        # ============= 5. 基礎數值欄位（如股價、指數） =============
        # 邏輯：採用時間序列最穩健的填補法。
        # 1. ffill (前向填補)：使用前一天的資料，這符合實際交易邏輯（不參考未來資訊）。
        # 2. bfill (後向填補)：若第一筆就缺值，才往後找。
        # 3. 0：最後的保底填補。
        if df[col].dtype != 'O':
            df[col] = df[col].fillna(method='ffill').fillna(method='bfill').fillna(0)

    return df

## ⚙️ Phase 3: Automated Pipeline & Temporal Alignment

To ensure that model training aligns with real-world trading logic and to improve development efficiency, we constructed the `preprocess_one_df` integration function. This phase focuses on addressing common issues in time-series data, such as **Data Leakage** and **Temporal Alignment**.

### 1. Target Labeling
* **Target Variable**: `Movement_tomorrow`.
* **Logic**: We use `.shift(-1)` to move the "next trading day" price movement label to the current row. This allows the model to learn how to predict future market directions using features known today.

### 2. Market Lag Correction
* **Background**: Due to time zone differences, when the Taiwan market opens, the closing information from the U.S. market for the previous night has just been finalized.
* **Alignment**: We apply a `.shift(1)` operation to the `US_FEATURES`. This ensures that during the training process, U.S. market features always lag the target label by one day, strictly adhering to backtesting principles that "only historical information is used to predict the future."

### 3. Pipeline Automation
This function encapsulates the following steps:
1. **Temporal Sorting**: Ensures all shift operations are based on the correct chronological order.
2. **Feature Engineering Integration**: Calls `enhance_features` to perform multi-dimensional indicator expansion.
3. **Missing Value Handling**: Executes `fill_na_safely` for logical data imputation.
4. **Invalid Sample Cleaning**: Removes invalid data points at the beginning and end of the dataset caused by `shift` operations (e.g., the last row, which cannot obtain "tomorrow's" answer).

In [8]:
def preprocess_one_df(df_raw: pd.DataFrame, is_train: int) -> pd.DataFrame:
    """
    單一資料集完整預處理流水線：
    包含：定義標籤、處理時差、特徵增強、缺失值清理。
    """
    df = df_raw.copy()
    df['is_train'] = is_train

    # 1. 時間排序：確保時間序列的連續性，這是所有 shift 操作的前提
    if 'Date' in df.columns:
        df = df.sort_values('Date').reset_index(drop=True)

    # 2. 定義預測目標 (Target)：
    # 使用 .shift(-1) 將「明天的漲跌」移到「今天」的列，讓模型學習用今天預測明天
    df['Movement_tomorrow'] = df['Movement'].shift(-1)

    # 3. 美股資訊延遲處理 (Market Lag Handling)：
    # 由於美股開盤時間晚於台股，台股當天開盤時只能參考到「美股昨晚」的表現。
    # 使用 .shift(1) 確保模型不會使用到「未來」的美股資訊，符合真實交易邏輯。
    for col in US_FEATURES:
        if col in df.columns:
            df[col] = df[col].shift(1)

    # 4. 特徵工程增強：執行自定義指標計算 (如 RSI, MACD, K線型態等)
    df = enhance_features(df)

    # 5. 智能 NA 填補：根據特徵屬性進行分類填補，最大化樣本利用率
    df = fill_na_safely(df)

    # 6. 清理無標籤樣本：
    # 由於 shift(-1)，最後一筆資料會沒有「明天的答案」，必須剔除以防干擾訓練。
    df = df.dropna(subset=['Movement_tomorrow']).reset_index(drop=True)

    # 7. 型態轉換：確保預測目標為分類格式 (int)
    df['Movement_tomorrow'] = df['Movement_tomorrow'].astype(int)

    return df

In [9]:
# ------------------------------------------------
# 4. 對 train / test 各自前處理（避免任何洩漏）
# ------------------------------------------------
train_proc = preprocess_one_df(train_raw, is_train=1)
test_proc  = preprocess_one_df(test_raw,  is_train=0)

## 🎯 Phase 4: Feature Selection & Cleaning

Before model training, we perform a final feature calibration to ensure the quality and stability of the input data.

### 1. Automated Feature Filtering
* **Exclude Non-Predictive Fields**: Remove `Date` (time-series label), `Movement_tomorrow` (target label), and the auxiliary `is_train` flag.
* **Type Filtering**: Automatically iterate through all columns to retain only numerical features, excluding string or object types to ensure compliance with Scikit-learn model input requirements.

### 2. Numerical Stability
* **Handling Infinite Values**: Financial indicators involving ratios (such as price fluctuations or volume ratios) may occasionally result in `inf` due to zero denominators. These are replaced with `0` to avoid errors during the subsequent scaling process.

In [10]:

# ------------------------------------------------
# 5. 定義特徵欄位
# ------------------------------------------------
exclude_cols = ['Date', 'Movement', 'Movement_tomorrow', 'is_train']
features = [
    c for c in train_proc.columns
    if c not in exclude_cols and train_proc[c].dtype != 'O'
]

X_train_full = train_proc[features]
y_train_full = train_proc['Movement_tomorrow']

X_test = test_proc[features]
y_test = test_proc['Movement_tomorrow']

# 避免 inf
X_train_full = X_train_full.replace([np.inf, -np.inf], 0)
X_test       = X_test.replace([np.inf, -np.inf], 0)

In [11]:
# 挑選幾個關鍵欄位來對齊看看
check_cols = ['Date', 'Movement', 'Movement_tomorrow']
print("--- 檢查標籤對齊 (今日 vs 明日) ---")
print(train_proc[check_cols].head(5))

print("\n--- 檢查美股特徵延遲 (Market Lag) ---")
# 假設妳原本讀入的原始資料叫 train_raw
print("原始第二筆 DJI:", train_raw['DJI'].iloc[2])
print("處理後第二筆 DJI (應為 NaN 或已填補):", train_proc['DJI'].iloc[2])
print("處理後第三筆 DJI (應等於原始第二筆):", train_proc['DJI'].iloc[3])

print("\n--- 檢查數值品質 ---")
print(f"訓練集總樣本數: {len(train_proc)}")
print(f"是否有遺失值 (NA): {train_proc[features].isna().any().any()}")
print(f"是否有無窮大 (inf): {np.isinf(train_proc[features]).any().any()}")
print(f"特徵總數: {len(features)}")

--- 檢查標籤對齊 (今日 vs 明日) ---
        Date  Movement  Movement_tomorrow
0 2010-01-04         0                  0
1 2010-01-05         0                  1
2 2010-01-06         1                  0
3 2010-01-07         0                  0
4 2010-01-08         0                  1

--- 檢查美股特徵延遲 (Market Lag) ---
原始第二筆 DJI: 10573.68
處理後第二筆 DJI (應為 NaN 或已填補): 10572.02
處理後第三筆 DJI (應等於原始第二筆): 10573.68

--- 檢查數值品質 ---
訓練集總樣本數: 3680
是否有遺失值 (NA): False
是否有無窮大 (inf): False
特徵總數: 117


## ⚖️ Phase 5: Time-Series Split

In financial forecasting models, the chronological order of samples is critical. To simulate a realistic investment environment, we abandon traditional random sampling in favor of a validation mechanism based on **chronological splitting**.

### 1. Splitting Logic
* **Training Set (Train)**: Utilizes the first 80% of the historical data (starting from 2010) for model parameter learning.
* **Validation Set (Validation)**: Uses the remaining 20% of the training data for hyperparameter fine-tuning (e.g., selecting the optimal K-value for KNN) before the final evaluation.
* **Test Set (Test)**: Maintains the final year of data as an independent dataset to serve as the benchmark for final performance evaluation.

### 2. No Shuffling Principle
This project strictly adheres to the "No Shuffling" principle. By ensuring that training data always precedes validation and testing data chronologically, we completely eliminate **Look-ahead Bias**, ensuring that our backtesting results possess genuine practical reference value.

In [12]:
# ------------------------------------------------
# 6. 時間序列切割 train / val（按時間順序）
# ------------------------------------------------
n_train = len(X_train_full)
split_idx = int(n_train * 0.8)

X_train = X_train_full.iloc[:split_idx]
y_train = y_train_full.iloc[:split_idx]

X_val   = X_train_full.iloc[split_idx:]
y_val   = y_train_full.iloc[split_idx:]

print(f"Train size = {len(X_train)}, Val size = {len(X_val)}, Test size = {len(X_test)}")

Train size = 2944, Val size = 736, Test size = 205


## 🧪 Phase 6: Hyperparameter Tuning - K-Sweep

To find the most suitable KNN model parameters for this dataset, we implemented the `sweep_k` function to perform a performance scan across different numbers of neighbors ($k$).

### 1. Evaluation Metrics
In addition to basic **Accuracy**, we specifically monitor the following metrics:
* **Recall (Class 1)**: In stock forecasting, the ability to capture "Upward" movements is crucial.
* **Macro F1-score**: This accounts for the balance between "Up" and "Down" classes, ensuring the model performs equally well for both directions without bias.

### 2. Scaling Consistency
During the sweep process, we strictly execute `fit_transform` on the training set and `transform` on the validation set. This ensures that the model testing environment maintains data scaling consistency identical to future real-world deployment.

### 3. Optimization Goal
Our objective is to select a $k$ value where both **Macro F1-score** and **Accuracy** are at high levels and the metric performance remains relatively stable, thereby enhancing the model's generalization capability.

In [13]:
def sweep_k(X_train, X_val, y_train, y_val, k_list=[3,5,7,11,15,21,31,41]):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled   = scaler.transform(X_val)

    results = []

    print("===== Sweep K =====")
    for k in k_list:
        knn = KNeighborsClassifier(
            n_neighbors=k,
            weights='uniform'
        )
        knn.fit(X_train_scaled, y_train)
        y_pred = knn.predict(X_val_scaled)

        acc = accuracy_score(y_val, y_pred)
        rec1 = recall_score(y_val, y_pred)
        f1m  = f1_score(y_val, y_pred, average='macro')

        print(f"k={k:2d} | Acc={acc:.4f} | Recall1={rec1:.4f} | MacroF1={f1m:.4f}")

        results.append((k, acc, rec1, f1m))

    return results


### 📊 Hyperparameter Tuning Result Analysis

Following the sweep of $k \in \{3, 5, \dots, 41\}$, we observed the following:
1. **Optimal Parameter**: At $k=11$, the model achieved the highest Accuracy and Macro F1 on the validation set, demonstrating an optimal balance between model complexity and generalization capability.
2. **Final Strategy**: We have selected $k=11$ as the definitive parameter for the KNN model to be applied for performance validation on the independent test set.

In [14]:
results = sweep_k(X_train, X_val, y_train, y_val)

===== Sweep K =====
k= 3 | Acc=0.4620 | Recall1=0.5050 | MacroF1=0.4581
k= 5 | Acc=0.4715 | Recall1=0.5176 | MacroF1=0.4673
k= 7 | Acc=0.5041 | Recall1=0.5704 | MacroF1=0.4977
k=11 | Acc=0.5408 | Recall1=0.6181 | MacroF1=0.5331
k=15 | Acc=0.5204 | Recall1=0.6106 | MacroF1=0.5108
k=21 | Acc=0.5054 | Recall1=0.6131 | MacroF1=0.4928
k=31 | Acc=0.4946 | Recall1=0.6357 | MacroF1=0.4750
k=41 | Acc=0.5122 | Recall1=0.6558 | MacroF1=0.4925


In [15]:
def run_basic_knn(X_train, X_val, y_train, y_val, n_neighbors=11):
    # 標準化
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled   = scaler.transform(X_val)

    # 建立最基礎的 KNN 模型
    knn = KNeighborsClassifier(
        n_neighbors=n_neighbors,
        weights='uniform',   # 先用 uniform，之後可以試 distance
        metric='minkowski',  # p=2 → Euclidean
        p=2
    )

    # 訓練
    knn.fit(X_train_scaled, y_train)

    # 預測
    y_pred = knn.predict(X_val_scaled)

    # 評估
    acc = accuracy_score(y_val, y_pred)
    rec_up = recall_score(y_val, y_pred, pos_label=1)
    rec_down = recall_score(y_val, y_pred, pos_label=0)
    f1_macro = f1_score(y_val, y_pred, average='macro')

    print("===== Basic KNN (k = {}) 評估結果 =====".format(n_neighbors))
    print(f"Accuracy        : {acc:.4f}")
    print(f"Recall (Up=1)   : {rec_up:.4f}")
    print(f"Recall (Down=0) : {rec_down:.4f}")
    print(f"Macro F1        : {f1_macro:.4f}")
    print("\nClassification Report:")
    print(classification_report(y_val, y_pred, digits=4))

    print("Confusion Matrix:")
    print(confusion_matrix(y_val, y_pred))

    return knn, scaler


In [16]:
knn_model, knn_scaler = run_basic_knn(X_train, X_val, y_train, y_val, n_neighbors=11)

===== Basic KNN (k = 11) 評估結果 =====
Accuracy        : 0.5408
Recall (Up=1)   : 0.6181
Recall (Down=0) : 0.4497
Macro F1        : 0.5331

Classification Report:
              precision    recall  f1-score   support

           0     0.5000    0.4497    0.4735       338
           1     0.5694    0.6181    0.5928       398

    accuracy                         0.5408       736
   macro avg     0.5347    0.5339    0.5331       736
weighted avg     0.5376    0.5408    0.5380       736

Confusion Matrix:
[[152 186]
 [152 246]]


In [17]:
X_test_scaled = knn_scaler.transform(X_test)
y_test_pred = knn_model.predict(X_test_scaled)
acc_test = accuracy_score(y_test, y_test_pred)
rec_up_test = recall_score(y_test, y_test_pred, pos_label=1)
rec_down_test = recall_score(y_test, y_test_pred, pos_label=0)
macro_f1_test = f1_score(y_test, y_test_pred, average='macro')

print("===== KNN Final Test Evaluation =====")
print(f"Accuracy     : {acc_test:.4f}")
print(f"Recall Up    : {rec_up_test:.4f}")
print(f"Recall Down  : {rec_down_test:.4f}")
print(f"Macro F1     : {macro_f1_test:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_test_pred, digits=4))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_test_pred))

===== KNN Final Test Evaluation =====
Accuracy     : 0.4585
Recall Up    : 0.4380
Recall Down  : 0.4881
Macro F1     : 0.4567

Classification Report:
              precision    recall  f1-score   support

           0     0.3761    0.4881    0.4249        84
           1     0.5521    0.4380    0.4885       121

    accuracy                         0.4585       205
   macro avg     0.4641    0.4631    0.4567       205
weighted avg     0.4800    0.4585    0.4624       205

Confusion Matrix:
[[41 43]
 [68 53]]


## 📉 Dimensionality Reduction via PCA

To enhance model efficiency and reduce high-dimensional noise, we implemented **Principal Component Analysis (PCA)**. This technique transforms the original 117 features into a set of linearly uncorrelated variables.

### 1. Motivation for PCA
* **Mitigating the Curse of Dimensionality**: With over 100 features, distance-based models like KNN often struggle as the feature space becomes sparse. PCA helps consolidate information into fewer dimensions.
* **Noise Reduction**: By retaining only the principal components with the highest variance, we filter out low-variance noise that may interfere with the model's predictive signals.

### 2. Implementation Strategy
* **Feature Scaling**: Since PCA is sensitive to the scale of the original variables, we apply `StandardScaler` prior to the transformation to ensure each feature contributes equally.
* **Component Selection**: We conducted a grid search across different component counts (e.g., 20, 30, 40) to determine the optimal balance between information retention and model performance.
* **Result**: Through empirical testing, **40 components** provided the best Macro F1-score, effectively capturing the core variance of the market data.

In [18]:
from sklearn.decomposition import PCA
def run_pca_knn(
    X_train, X_val, y_train, y_val,
    n_components=30,   # PCA 主成分數量
    k_neighbors=11     # KNN 的 k
):
    from sklearn.preprocessing import StandardScaler
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.metrics import accuracy_score, recall_score, f1_score

    # ======= 1. 標準化（PCA & KNN 都需要）=======
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled   = scaler.transform(X_val)

    # ======= 2. PCA 降維 =======
    pca = PCA(n_components=n_components, random_state=42)
    X_train_pca = pca.fit_transform(X_train_scaled)
    X_val_pca   = pca.transform(X_val_scaled)

    # ======= 3. KNN =======
    knn = KNeighborsClassifier(
        n_neighbors=k_neighbors,
        weights='distance'
    )
    knn.fit(X_train_pca, y_train)

    # ======= 4. 預測 =======
    y_pred = knn.predict(X_val_pca)

    # ======= 5. 評估 =======
    acc = accuracy_score(y_val, y_pred)
    rec1 = recall_score(y_val, y_pred)
    rec0 = recall_score(y_val, y_pred, pos_label=0)
    f1m  = f1_score(y_val, y_pred, average='macro')

    #print(f"===== PCA + KNN (PCA={n_components}, k={k_neighbors}) =====")
    #print(f"Accuracy        : {acc:.4f}")
    #print(f"Recall (Up=1)   : {rec1:.4f}")
    #print(f"Recall (Down=0) : {rec0:.4f}")
    #print(f"Macro F1        : {f1m:.4f}")
    #print("\nClassification Report:")
    #print(classification_report(y_val, y_pred, digits=4))

    return {
        'acc': acc,
        'recall_up': rec1,
        'recall_down': rec0,
        'macro_f1': f1m
    }


In [19]:
def search_best_pca_knn(
    X_train, X_val, y_train, y_val,
    pca_list=[10, 20, 30, 40, 50],
    k_list=[3, 5, 11, 15, 21, 31, 41, 51]
):
    best_f1 = -1
    best_params = None
    best_model = None

    for pca_n in pca_list:
        for k in k_list:
            result = run_pca_knn(
                X_train, X_val, y_train, y_val,
                n_components=pca_n,
                k_neighbors=k
            )

            if result["macro_f1"] > best_f1:
                best_f1 = result["macro_f1"]
                best_params = (pca_n, k)
                best_model = result

    print("\n===== BEST MODEL =====")
    print(f"PCA components : {best_params[0]}")
    print(f"K neighbors    : {best_params[1]}")
    print(f"Best Macro F1  : {best_f1:.4f}")

    return best_params

In [20]:
best_pca, best_k = search_best_pca_knn(
    X_train, X_val, y_train, y_val,
    pca_list=[20,30,40],
    k_list=[5,11,21,31]
)


===== BEST MODEL =====
PCA components : 40
K neighbors    : 11
Best Macro F1  : 0.5129


In [21]:
pca = PCA(n_components=best_pca)
knn = KNeighborsClassifier(n_neighbors=best_k, weights='distance')

In [22]:
# 1. scale again
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_full)
X_test_scaled  = scaler.transform(X_test)

# 2. PCA again
pca = PCA(n_components=best_pca)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca  = pca.transform(X_test_scaled)

# 3. Train KNN
knn = KNeighborsClassifier(n_neighbors=best_k, weights='distance')
knn.fit(X_train_pca, y_train_full)

# 4. Predict test
y_test_pred = knn.predict(X_test_pca)

# 5. Evaluate test
acc_test = accuracy_score(y_test, y_test_pred)
rec_up_test = recall_score(y_test, y_test_pred, pos_label=1)
rec_down_test = recall_score(y_test, y_test_pred, pos_label=0)
macro_f1_test = f1_score(y_test, y_test_pred, average='macro')

print("===== KNN(after PCA) Final Test Evaluation =====")
print(f"Accuracy     : {acc_test:.4f}")
print(f"Recall Up    : {rec_up_test:.4f}")
print(f"Recall Down  : {rec_down_test:.4f}")
print(f"Macro F1     : {macro_f1_test:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_test_pred, digits=4))

===== KNN(after PCA) Final Test Evaluation =====
Accuracy     : 0.5854
Recall Up    : 0.7438
Recall Down  : 0.3571
Macro F1     : 0.5465

Classification Report:
              precision    recall  f1-score   support

           0     0.4918    0.3571    0.4138        84
           1     0.6250    0.7438    0.6792       121

    accuracy                         0.5854       205
   macro avg     0.5584    0.5505    0.5465       205
weighted avg     0.5704    0.5854    0.5705       205



In [23]:
pca40 = PCA(n_components=40)
X_train_40 = pca40.fit_transform(X_train)
X_train_full_40 = pca40.fit_transform(X_train_full)
X_val_40   = pca40.transform(X_val)
X_test_40  = pca40.transform(X_test)

## 🧬 Phase 7: Advanced Feature Engineering - Cluster-Augmented KNN

To further capture the non-linear structures and various market states within financial data, we implemented the `run_cluster_knn` function. This represents an innovative approach that combines unsupervised learning with supervised learning.

### 1. Core Logic: Spatial Dimension Augmentation
* **K-Means Market Clustering**: Following PCA dimensionality reduction, we utilize the K-Means algorithm to cluster the compressed feature space. This effectively identifies specific "Market Regimes," such as volatile periods or trending phases defined by specific combinations of technical indicators.
* **Feature Augmentation**: The Cluster Labels generated by K-Means are appended as a new dimension to the feature matrix. This guides the KNN model to prioritize historical samples belonging to the same "Market State" when identifying nearest neighbors.



### 2. Model Robustness Mechanisms
* **Temporal Consistency**: To strictly adhere to backtesting logic, the K-Means model is only "fitted" on the training set (`X_train`), while the validation and test sets are processed in "predict" mode only.
* **Multi-dimensional Metric Evaluation**: Given the inherent difficulty of financial forecasting, we simultaneously monitor **Recall Up** (bullish accuracy) and **Recall Down** (bearish accuracy) to ensure the model maintains an unbiased predictive stance.

### 3. Motivation for Optimization
Even after PCA reduces the features to 40 dimensions, KNN can still struggle with subtle boundary definitions. By introducing the Cluster Label as a "logical anchor" in the distance calculation formula, we provide the model with a clear environmental context, thereby enhancing the resolution of the classification.

In [24]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, recall_score, f1_score

def run_cluster_knn(X_train, X_val, y_train, y_val, 
                    n_clusters=7, n_neighbors=11):

    # 先用同一個 scaler 做 KMeans & KNN
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled   = scaler.transform(X_val)

    # KMeans clustering（只 fit 在 train）
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_train = kmeans.fit_predict(X_train_scaled)
    cluster_val   = kmeans.predict(X_val_scaled)

    # 將 cluster label 加入特徵
    X_train_aug = np.column_stack([X_train_scaled, cluster_train])
    X_val_aug   = np.column_stack([X_val_scaled, cluster_val])

    # 訓練 KNN
    knn = KNeighborsClassifier(
        n_neighbors=n_neighbors,
        weights='uniform',
        metric='minkowski',
        p=2
    )
    knn.fit(X_train_aug, y_train)

    # 預測
    y_pred = knn.predict(X_val_aug)

    # 評估
    acc = accuracy_score(y_val, y_pred)
    rec_up = recall_score(y_val, y_pred, pos_label=1)
    rec_down = recall_score(y_val, y_pred, pos_label=0)
    macro_f1 = f1_score(y_val, y_pred, average='macro')

    print(f"\n===== ClusterKNN: clusters={n_clusters}, k={n_neighbors} =====")
    print(f"Accuracy     : {acc:.4f}")
    print(f"Recall Up    : {rec_up:.4f}")
    print(f"Recall Down  : {rec_down:.4f}")
    print(f"Macro F1     : {macro_f1:.4f}")

    return {
        "model": knn,
        "scaler": scaler,
        "kmeans": kmeans,
        "val_macro_f1": macro_f1
    }


In [25]:
for c in [3, 5, 7, 10, 15]:
    out = run_cluster_knn(
        X_train_40, X_val_40, y_train, y_val,
        n_clusters=c, 
        n_neighbors=11
    )


===== ClusterKNN: clusters=3, k=11 =====
Accuracy     : 0.5707
Recall Up    : 0.7312
Recall Down  : 0.3817
Macro F1     : 0.5488

===== ClusterKNN: clusters=5, k=11 =====
Accuracy     : 0.5720
Recall Up    : 0.7236
Recall Down  : 0.3935
Macro F1     : 0.5521

===== ClusterKNN: clusters=7, k=11 =====
Accuracy     : 0.5774
Recall Up    : 0.7387
Recall Down  : 0.3876
Macro F1     : 0.5557

===== ClusterKNN: clusters=10, k=11 =====
Accuracy     : 0.5679
Recall Up    : 0.7211
Recall Down  : 0.3876
Macro F1     : 0.5476

===== ClusterKNN: clusters=15, k=11 =====
Accuracy     : 0.5598
Recall Up    : 0.7211
Recall Down  : 0.3698
Macro F1     : 0.5374


## 🏁 Phase 8: Final Model Evaluation

After completing hyperparameter tuning, we merge the training and validation sets to retrain the optimized **Cluster-Augmented KNN** model using the maximum available historical data. A final performance evaluation is then conducted on the independent test set (2024 data).

### 1. Final Model Configuration
* **Feature Processing**: Integrates `StandardScaler` normalization with 117-dimensional enhanced features.
* **Clustering Strategy**: Employs K-Means clustering with `n_clusters=7` to provide the model with essential market regime context.
* **Neighbor Voting**: Configured with `n_neighbors=11` to maintain sensitivity to local features while ensuring robust noise filtration.

### 2. Multi-dimensional Evaluation Metrics
To comprehensively analyze the model's practical utility, we focus on several key metrics beyond basic **Accuracy**:
* **Confusion Matrix**: Provides a direct visualization of the distribution between "Actual Up/Down" and "Predicted Up/Down" to identify potential model biases.
* **Recall Up/Down**: Measures the model's sensitivity in capturing both upward and downward market trends.
* **Classification Report**: Provides Precision and F1-score to quantify the overall stability and reliability of the predictions.

In [26]:
def final_cluster_knn(X_train_full, y_train_full, X_test, y_test,
                      n_clusters=7, n_neighbors=11):
    print("===== Training Final Cluster-KNN Model =====")

    # StandardScaler（fit 在全部 train）
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_full)
    X_test_scaled  = scaler.transform(X_test)

    # KMeans（fit 在全部 train）
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_train = kmeans.fit_predict(X_train_scaled)
    cluster_test  = kmeans.predict(X_test_scaled)

    # 將 cluster label 加入特徵
    X_train_aug = np.column_stack([X_train_scaled, cluster_train])
    X_test_aug  = np.column_stack([X_test_scaled, cluster_test])

    # KNN 訓練
    knn = KNeighborsClassifier(
        n_neighbors=n_neighbors,
        weights='uniform',
        metric='minkowski',
        p=2
    )
    knn.fit(X_train_aug, y_train_full)

    # Test 預測
    y_pred_test = knn.predict(X_test_aug)

    # 評估
    acc = accuracy_score(y_test, y_pred_test)
    rec_up = recall_score(y_test, y_pred_test, pos_label=1)
    rec_down = recall_score(y_test, y_pred_test, pos_label=0)
    macro_f1 = f1_score(y_test, y_pred_test, average='macro')

    print("\n===== Final Cluster-KNN Test Evaluation =====")
    print(f"Accuracy     : {acc:.4f}")
    print(f"Recall Up    : {rec_up:.4f}")
    print(f"Recall Down  : {rec_down:.4f}")
    print(f"Macro F1     : {macro_f1:.4f}")

    print("\nClassification Report:")
    print(classification_report(y_test, y_pred_test, digits=4))

    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred_test))

    return {
        "knn": knn,
        "kmeans": kmeans,
        "scaler": scaler,
        "macro_f1_test": macro_f1
    }


In [27]:
result = final_cluster_knn(
    X_train_full_40, y_train_full,
    X_test_40, y_test,
    n_clusters=7,
    n_neighbors=11
)

===== Training Final Cluster-KNN Model =====

===== Final Cluster-KNN Test Evaluation =====
Accuracy     : 0.6390
Recall Up    : 0.5455
Recall Down  : 0.7738
Macro F1     : 0.6390

Classification Report:
              precision    recall  f1-score   support

           0     0.5417    0.7738    0.6373        84
           1     0.7765    0.5455    0.6408       121

    accuracy                         0.6390       205
   macro avg     0.6591    0.6596    0.6390       205
weighted avg     0.6803    0.6390    0.6393       205

Confusion Matrix:
[[65 19]
 [55 66]]


## 📈 Phase 9: Probability Estimation & Decision Support

Following the final evaluation, the model is applied to the test set to generate not only binary class predictions but also specific **Upward Probabilities**. This allows us to quantify prediction confidence, serving as a critical filter for real-world trading execution.

### 1. Cross-Model Consistency Transformation
* **Standardization Application**: The `scaler` generated during the training phase is strictly applied to the test set to ensure consistency across the data space.
* **Cluster Mapping**: The pre-trained `K-Means` model is utilized to identify the market regime (Cluster Label) for each sample in the test set.

### 2. From "Classification" to "Probability" (predict_proba)
* **Confidence Quantization**: We extract the probability of an "Upward (Class 1)" movement using the `predict_proba(X_test_aug)[:, 1]` function.
* **Decision Thresholds**: 
    * A probability near 1.0 indicates a strong bullish consensus among nearest neighbors, representing high confidence.
    * A probability near 0.5 suggests a market "tug-of-war" between bulls and bears, signaling a need for cautious entry.

In [31]:
# 1. 用 scaler + kmeans 處理 test 資料
X_test_scaled = result['scaler'].transform(X_test_40)
cluster_test = result['kmeans'].predict(X_test_scaled)

# 2. 增加 cluster label
X_test_aug = np.column_stack([X_test_scaled, cluster_test])

# 3. 產生上漲機率
y_pred_prob = result['knn'].predict_proba(X_test_aug)[:, 1]

## 💾 Phase 10: Final Output Generation

In this phase, the model's prediction results for the 2025 test set are exported into a CSV format. This file serves as the core dataset for subsequent strategy backtesting and live demonstrations.

### 1. Output Field Definitions
* **Date**: The trading date corresponding to the test set.
* **Predict**: Stores the model's predicted "Upward Probability." Compared to binary classification (0/1), probability values provide more granular decision support—for instance, a probability higher than 0.7 can be treated as a strong buy signal.

### 2. Data Encapsulation and Persistence
By aligning the prediction results with timestamps, we ensure the connectivity between the model output and original stock price data. This establishes a solid foundation for upcoming model demonstrations and performance analysis.

In [29]:
import os

# 1. 定義輸出路徑
output_dir = "../../../data/results/KNN"

# 2. 如果資料夾不存在，則自動建立
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"📁 已建立資料夾：{output_dir}")

# 3. 建立輸出 DataFrame
output_2025 = pd.DataFrame({
    "Date": test_proc["Date"].values,
    "Predict": y_pred_prob
})

# 4. 輸出成 CSV（放到指定資料夾下）
output_path = os.path.join(output_dir, "KNN_prediction_2022.csv")
output_2025.to_csv(output_path, index=False)

print(f"📄 已輸出：{output_path}")

📄 已輸出：../../../data/results/KNN\KNN_prediction_2022.csv


## 📂 Phase 11: Dataset Export & Multi-Year Prediction Merging

In the final stage of this project, we perform the physical partitioning of data and the automated aggregation of prediction results to ensure that the model output meets the requirements for system deployment.

### 1. Temporal Dataset Partitioning
* **Training and Test Sets**: Data is strictly partitioned by year (e.g., using 2022 as a boundary) to ensure the validation process adheres to a realistic chronological progression.
* **Persistent Storage**: The partitioned files, such as `2022_train.csv` and `2022_test.csv`, are exported to facilitate in-depth backtesting for specific years.



> ⚠️ **Stacking Preparation Reminder**:  
> To prevent **Data Leakage**, a "Rolling Forecast" approach must be adopted when generating Meta-features for the Stacking ensemble layer:
> 1. First, generate physical data slices (train/test) for a specific year (e.g., 2020).
> 2. Direct the preprocessing pipeline to that slice, retrain the model, and export `KNN_prediction_2020.csv`.
> 3. Repeat this process annually (2020–2024), and finally use the `glob` module to perform automated merging.

In [216]:
df = pd.read_csv("../../../data/processed/train.csv")
df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values("Date").reset_index(drop=True)

In [217]:
df_train_2020 = df[df["Date"].dt.year < 2024]

In [218]:
df_test_2020 = df[df["Date"].dt.year == 2024]

In [219]:
# 1. 定義處理後資料的存放路徑
processed_dir = "../../../data/processed"

# 2. 檢查並建立資料夾
if not os.path.exists(processed_dir):
    os.makedirs(processed_dir)

# 3. 定義完整的檔案路徑
train_path_2020 = os.path.join(processed_dir, "2024_train.csv")
test_path_2020  = os.path.join(processed_dir, "2024_test.csv")

# 4. 儲存檔案
df_train_2020.to_csv(train_path_2020, index=False)
df_test_2020.to_csv(test_path_2020, index=False)

print(f"✅ 已儲存至：{processed_dir}")

✅ 已儲存至：../../data/processed


### 2. Automated Result Merging
* **Batch Retrieval**: Utilizes the `glob` module to automatically detect all files matching the `KNN_prediction_*.csv` pattern, enhancing code scalability.
* **Temporal Concatenation**: Yearly prediction probabilities are vertically concatenated and re-sorted by date. This ensures the final `KNN_prediction_all.csv` maintains a complete and continuous time series.
* **Final Output**: This consolidated file serves as the core input for the subsequent Stacking ensemble layer.

In [604]:
import glob

# 1. 定義路徑
input_pattern = "../../../data/results/KNN/KNN_prediction_*.csv" # 從 results 讀取各年份檔案
output_dir = "../../../data/results/KNN"
output_file = "KNN_prediction_all.csv"

# 2. 檢查並建立輸出資料夾
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 3. 尋找所有預測檔案
files = sorted(glob.glob(input_pattern))

if not files:
    print("❌ 找不到預測檔案，請確認 data/results/ 下是否有檔案。")
else:
    dfs = []
    for f in files:
        df = pd.read_csv(f)
        dfs.append(df)

    # 合併數據
    merged = pd.concat(dfs, ignore_index=True)

    # 若有 Date 欄位則進行排序
    if "Date" in merged.columns:
        merged["Date"] = pd.to_datetime(merged["Date"])
        merged = merged.sort_values("Date").reset_index(drop=True)

    # 4. 輸出到 data/processed/KNN/
    final_save_path = os.path.join(output_dir, output_file)
    merged.to_csv(final_save_path, index=False)

    print(f"✔ 已輸出最終合併檔：{final_save_path}")

✔ 已輸出最終合併檔：../../data/results/KNN\KNN_prediction_all.csv
